In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import os

In [2]:
df = pd.read_csv('../data_final/data.csv')

In [12]:
# Função para tratar outliers
def tratar_outliers(df, colunas):
    df_out = df.copy()
    for coluna in colunas:
        Q1 = df_out[coluna].quantile(0.25)
        Q3 = df_out[coluna].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        df_out = df_out[(df_out[coluna] >= limite_inferior) & (df_out[coluna] <= limite_superior)]
    return df_out

# Função para calcular a acurácia percentual média
def calcular_acuracia(y_real, y_previsto):
    acuracias = 1 - np.abs((y_real - y_previsto) / y_real)
    acuracias = acuracias[y_real != 0]
    return np.mean(acuracias) * 100

# Categorias da Uber
uber_categories = ['UberX', 'Comfort', 'Black']
df_uber = df[df['ProductID'].isin(uber_categories)]

# Features e target
features = ['distance_km', 'distance_pico', 'effective_distance']
target = 'Price'


model_dir = './model'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
else:
    for file in os.listdir(model_dir):
        file_path = os.path.join(model_dir, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)

In [ ]:
"""# Parâmetros para RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# Treinamento por categoria com ajuste de hiperparâmetros
for category in uber_categories:
    print(f"\n--- Treinando modelo Random Forest para: {category} ---")
    
    df_category = df_uber[df_uber['ProductID'] == category]
    df_category = tratar_outliers(df_category, features + [target])
    
    X = df_category[features]
    y = df_category[target]
    
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    model = RandomForestRegressor(random_state=42)
    
    # Ajuste de hiperparâmetros com RandomizedSearchCV
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=20, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
    random_search.fit(X_train, y_train)
    
    best_model = random_search.best_estimator_

    
    # Validação cruzada com o melhor modelo encontrado
    cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_rmse_scores = np.sqrt(-cv_scores)
    
    print(f"Validação Cruzada RMSE: {cv_rmse_scores.mean():.2f} (+/- {cv_rmse_scores.std():.2f})")

    
    y_pred_train = best_model.predict(X_train)
    y_pred_test = best_model.predict(X_test)
    
    mae_train = mean_absolute_error(y_train, y_pred_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    r2_train = r2_score(y_train, y_pred_train)
    
    mae_test = mean_absolute_error(y_test, y_pred_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    r2_test = r2_score(y_test, y_pred_test)
    
    acc_train = calcular_acuracia(y_train, y_pred_train)
    acc_test = calcular_acuracia(y_test, y_pred_test)
    
    print(f"Melhores Parâmetros: {random_search.best_params_}")
    print(f"Treinamento - MAE: {mae_train:.2f}, RMSE: {rmse_train:.2f}, R²: {r2_train:.2f}, Acurácia: {acc_train:.2f}%")
    print(f"Teste - MAE: {mae_test:.2f}, RMSE: {rmse_test:.2f}, R²: {r2_test:.2f}, Acurácia: {acc_test:.2f}%")
"""


--- Treinando modelo Random Forest para: UberX ---
Validação Cruzada RMSE: 5.11 (+/- 0.03)
Melhores Parâmetros: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
Treinamento - MAE: 2.97, RMSE: 4.20, R²: 0.82, Acurácia: 84.58%
Teste - MAE: 3.56, RMSE: 5.09, R²: 0.73, Acurácia: 81.66%

--- Treinando modelo Random Forest para: Comfort ---
Validação Cruzada RMSE: 7.88 (+/- 0.07)
Melhores Parâmetros: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
Treinamento - MAE: 4.28, RMSE: 6.28, R²: 0.85, Acurácia: 84.96%
Teste - MAE: 5.22, RMSE: 7.76, R²: 0.77, Acurácia: 81.79%

--- Treinando modelo Random Forest para: Black ---
Validação Cruzada RMSE: 7.27 (+/- 0.09)
Melhores Parâmetros: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
Treinamento - MAE: 3.18, RMSE: 4.84, R²: 0.96, Acurácia: 92.26%
Teste - MAE: 4.43, RM

--- Treinando modelo Random Forest para: UberX ---
Validação Cruzada RMSE: 5.11 (+/- 0.03)
Melhores Parâmetros: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
Treinamento - MAE: 2.97, RMSE: 4.20, R²: 0.82, Acurácia: 84.58%
Teste - MAE: 3.56, RMSE: 5.09, R²: 0.73, Acurácia: 81.66%

--- Treinando modelo Random Forest para: Comfort ---
Validação Cruzada RMSE: 7.88 (+/- 0.07)
Melhores Parâmetros: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
Treinamento - MAE: 4.28, RMSE: 6.28, R²: 0.85, Acurácia: 84.96%
Teste - MAE: 5.22, RMSE: 7.76, R²: 0.77, Acurácia: 81.79%

--- Treinando modelo Random Forest para: Black ---
Validação Cruzada RMSE: 7.27 (+/- 0.09)
Melhores Parâmetros: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
Treinamento - MAE: 3.18, RMSE: 4.84, R²: 0.96, Acurácia: 92.26%
Teste - MAE: 4.43, RMSE: 7.01, R²: 0.92, Acurácia: 89.35%

In [15]:
# Parâmetros fixos para o modelo
fixed_params = {
    'n_estimators': 100,
    'max_depth': 20,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'sqrt'
}

# Treinamento por categoria com parâmetros fixos
for category in uber_categories:
    print(f"\n--- Treinando modelo Random Forest para: {category} ---")
    
    df_category = df_uber[df_uber['ProductID'] == category]
    df_category = tratar_outliers(df_category, features + [target])
    
    X = df_category[features]
    y = df_category[target]
    
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    model = RandomForestRegressor(**fixed_params, random_state=42)
    model.fit(X_train, y_train)

    # Validação cruzada
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_rmse_scores = np.sqrt(-cv_scores)
    
    print(f"Validação Cruzada RMSE: {cv_rmse_scores.mean():.2f} (+/- {cv_rmse_scores.std():.2f})")

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    mae_train = mean_absolute_error(y_train, y_pred_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    r2_train = r2_score(y_train, y_pred_train)
    
    mae_test = mean_absolute_error(y_test, y_pred_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    r2_test = r2_score(y_test, y_pred_test)
    
    acc_train = calcular_acuracia(y_train, y_pred_train)
    acc_test = calcular_acuracia(y_test, y_pred_test)
    
    print(f"Treinamento - MAE: {mae_train:.2f}, RMSE: {rmse_train:.2f}, R²: {r2_train:.2f}, Acurácia: {acc_train:.2f}%")
    print(f"Teste - MAE: {mae_test:.2f}, RMSE: {rmse_test:.2f}, R²: {r2_test:.2f}, Acurácia: {acc_test:.2f}%")

    # Salvar modelo e scaler
    joblib.dump(model, os.path.join(model_dir, f"modelo_{category.lower()}.joblib"))
    joblib.dump(scaler, os.path.join(model_dir, f"scaler_{category.lower()}.joblib"))
    
    # Verificação
    if os.path.exists(os.path.join(model_dir, f"modelo_{category.lower()}.joblib")):
        print(f"Modelo para {category} salvo com sucesso.")
    else:
        print(f"Falha ao salvar modelo para {category}.")



--- Treinando modelo Random Forest para: UberX ---
Validação Cruzada RMSE: 5.11 (+/- 0.03)
Treinamento - MAE: 2.97, RMSE: 4.20, R²: 0.82, Acurácia: 84.58%
Teste - MAE: 3.56, RMSE: 5.09, R²: 0.73, Acurácia: 81.66%
Modelo para UberX salvo com sucesso.

--- Treinando modelo Random Forest para: Comfort ---
Validação Cruzada RMSE: 7.88 (+/- 0.07)
Treinamento - MAE: 4.28, RMSE: 6.28, R²: 0.85, Acurácia: 84.96%
Teste - MAE: 5.22, RMSE: 7.76, R²: 0.77, Acurácia: 81.79%
Modelo para Comfort salvo com sucesso.

--- Treinando modelo Random Forest para: Black ---
Validação Cruzada RMSE: 7.27 (+/- 0.09)
Treinamento - MAE: 3.18, RMSE: 4.84, R²: 0.96, Acurácia: 92.26%
Teste - MAE: 4.43, RMSE: 7.01, R²: 0.92, Acurácia: 89.35%
Modelo para Black salvo com sucesso.


## Melhorias

- UberX:
O modelo tem desempenho razoável e consistente entre treino e teste.

R² de 0.73 no teste indica que ele explica 73% da variabilidade — bom, mas com espaço para melhoria.

O RMSE de validação é bem próximo ao do teste, sugerindo boa generalização.


- Comfort:
O desempenho é razoavelmente bom, mas:

Há um leve overfitting: o modelo performa melhor no treino do que no teste.

A diferença entre RMSE de treino e teste sugere que ele aprendeu padrões específicos demais do conjunto de treinamento.


- Black:
Esse modelo tem desempenho excelente.

R² de 0.92 no teste é muito alto, indicando que o modelo quase explica toda a variabilidade.

Pequena diferença entre treino e teste → bom poder de generalização, com baixa variância.

Feature de distância, calculo está sendo feito de um ponto ao outro, sem considerar as ruas (o que está dando uma diferença), fazer o calculo mais preciso pode ajudar.